In [1]:
class EUtilities:
        
    def build_dataset(df, window, binary_target=False, delete_constant_values=True, PNL=False):
        """
        función para construir un data set
        window: tamaño de la ventana a utilizar para construir el dataset
        df: dataframe, con columna bid y ask.
        binary_target: si desea clasificar, este arroja 2 si el valor se mantiene,
        1 si el valor sube y 0 si este baja.
        delete_constant_values: default: True, elimina los valores que se mantienen

        retorna:
        X: dataset, con columna de PNL si así se especifica(default: False)
        y: target
        bt: binary target, default: False
        """

        import pandas as pd
        import numpy as np
        result = []
        binary = [] #para la columna objetivo binaria
        pnl_buy = [] #almacenar el pnl en caso de compra
        pnl_sell = [] #almacenar el pnl en caso de venta
        signal = df.bid
        ask = df.ask
        indx = signal.index[window-1:-1] #se toman los indicen que quedarán al final
        for i in range(len(signal)-window):

            if delete_constant_values == True:
                if signal[i+window] != signal[i+window-1]:

                    result.append(signal[i: i + window+1])
                    if PNL == True: 
                        pnl_sell.append(signal[i+window-1] - ask[i+window]) #calcular pnl en caso de venta-compra
                        pnl_buy.append(signal[i+window] - ask[i+window-1]) #calcular pnl en caso de compra-venta

                    if binary_target == True:
                        if signal[i+window] < signal[i+window-1]: 
                            if (signal[i+window-1] - ask[i+window]) > 0: binary.append(0) # 0 si baja y gano
                            else: binary.append(2) # 2 si baja pero pierdo

                        if signal[i+window] > signal[i+window-1]: # 1 si sube
                            if (signal[i+window] - ask[i+window-1]) > 0: binary.append(1) # 1 si sube y gano
                            else: binary.append(3) # 3 si sube pero pierdo

                else: indx = indx.delete(len(result))

            else:

                result.append(signal[i: i + window+1])
                if PNL == True:
                    pnl_sell.append(signal[i+window-1] - ask[i+window]) #calcular pnl en caso de venta-compra
                    pnl_buy.append(signal[i+window] - ask[i+window-1]) #calcular pnl en caso de compra-venta

            if binary_target == True and delete_constant_values == False:
                if signal[i+window] == signal[i+window-1]: binary.append(4) # 2 si se mantiene

                if signal[i+window] < signal[i+window-1]:
                    if (signal[i+window-1] - ask[i+window]) > 0: binary.append(0) # 0 si baja y gano
                    else: binary.append(2) # 2 si baja pero pierdo

                if signal[i+window] > signal[i+window-1]:
                    if (signal[i+window] - ask[i+window-1]) > 0: binary.append(1) # 1 si sube y gano
                    else: binary.append(3) # 3 si sube pero pierdo

        data = pd.DataFrame(np.array(result), index=indx)
        y = np.array(data.iloc[:,window])
        data = data.drop(window,axis=1)
        if PNL == True:
            data['PNL_0'] = pnl_sell
            data['PNL_1'] = pnl_buy 
        if binary_target == True: return data, y, np.array(binary)
        else: return data, y
    
    #para crear señales multiples
    
    def multi_signal(s_A, s_B):
        """
        dada dos señales s_A y s_B, se obtiene una multiseñal donde la
        señal que predomina para la contrucción es s_A
        """
        import pandas as pd
        import numpy as np
        #saco los valores que si están en el mismo instante de tiempo
        #se filtra la señal s_B por los indices que tiene s_A
        values = s_B.filter(items=s_A.index, axis=0)
        #inserto una columa llena de ceros para posteriormente colocar ahi los valores a utilizar
        s_A.insert(loc=2, column=s_B.columns[0]+'new', value=np.zeros((s_A.shape[0],1)))
        #llenamos con los datos
        s_A.loc[values.index,s_B.columns[0]+'new'] = values.loc[values.index,values.columns[0]]
        #llenamos los ceros
        s_A.loc[s_A.loc[s_A.index,s_B.columns[0]+'new']==0, s_B.columns[0]+'new'] = s_A.loc[s_A.loc[s_A.index,s_B.columns[0]+'new']==0,s_A.columns[0]]

        return s_A

    #redimensionar la salida del score
    
    def redim(signal):
        '''
        recibe un array plano el cual convierte en dos salidas,
        en una de estas se encuentran los resultados pertinentes
        a medir con el PNL y en el otro las predicciones.
        '''
        import numpy as np
        result = signal[:,0][0]
        predict = np.array(signal[:,1][0],dtype=np.float16)
        for i in range(1,len(signal)):
            result = np.concatenate((result,signal[:,0][i]))
            predict = np.concatenate((predict,signal[:,1][i]))
        result = result.reshape((len(signal),signal[:,0][0].shape[0]))
        return result, predict

   #para hacer una validación por pasos
    
    def step_validation(estimator, X, y, cv):
        '''
        Recibe el estimador,X,y, y un generador cv con el cual hace la validación
        dependiendo que la configuración que este tenga
        '''
        import numpy as np
        result = []
        for index in cv:
            estimator.fit(X.iloc[index[0]], y[index[0]])
            result.append(estimator.score(X.iloc[index[1]], y[index[1]]))
        return np.array(result)

    #particiona el dataset para sacar los indices para la validación por pasos
    
    def v_split(X, n_bdtrain, n_bdtest, mday):

        """"
        Hace un particionado del dataset, para tomar n_bdtrain días para entrenar
        y n_bdtest para probar, además, mday representa el paso de día a correr.
        X, dataframe, se necesita el indice de este para separar por días.
        n_bdtrain, número de bussines day utilizados para train.
        n_bdtest, número de bussines day utilizados para test.
        mday, días a correr para cada validación.

        """
        import pandas as pd
        import numpy as np
        from datetime import date
        start_day = 0

        #Divide el data set según días de train, test y cuanto se va moviendo
        bussines_day = []
        dates = pd.unique(X.index.date) #saco las fechas para luego tomar solo año-mes-día

        for i in dates: bussines_day.append(date.__format__(i,'%Y-%m-%d')) #lista de los bussines day

        intervals = []
        count = 0
        for i in bussines_day:
            f = len(X[i])-1 +count
            intervals.append([count,f])
            count = f+1

        for i in range(len(intervals)-n_bdtrain):
            yield(np.arange(intervals[start_day:start_day+n_bdtrain][0][0],
                            intervals[start_day:start_day+n_bdtrain][n_bdtrain-1][1]+1),
                  np.arange(intervals[n_bdtrain+start_day:n_bdtrain+start_day+n_bdtest][0][0],
                            intervals[n_bdtrain+start_day:n_bdtrain+start_day+n_bdtest][n_bdtest-1][1]+1))

            start_day += mday
            if start_day+n_bdtest > len(intervals)-n_bdtrain:
                break


In [35]:
%run code/PNLEstimatorWrapper.py

In [23]:
obj = EUtilities

In [40]:
#Se lee la señal correspondiente a GBP vs USD para el mes de enero de 2018
data_GBPvsUSD = pd.read_csv("./data/TrueFX/GBP-USD/GBPUSD-preprocesado-2018-01.csv", index_col=0
                            ,infer_datetime_format=True, parse_dates=True )

#Se lee la señal correspondiente a EUR vs USD para el mes de enero de 2018
data_EURvsUSD = pd.read_csv("./data/TrueFX/EUR-USD/datos_procesados_EURUSD-2018-01.csv", index_col=0
                            ,infer_datetime_format=True, parse_dates=True )

In [41]:
data_GBPvsUSD = obj.multi_signal(data_GBPvsUSD,data_EURvsUSD)

In [42]:
#los datos de el par GBP/USD para luego unirlos y formar los datos X para entrenar
X_GBPUSD, y_GBPUSD, bt_GBPUSD = obj.build_dataset(data_GBPvsUSD.iloc[:,0],window=3,binary_target=True,PNL=True)
#los datos de el par EUR/USD ""
X_EURUSD,_ = obj.build_dataset(data_GBPvsUSD.iloc[:,2],window=3,delete_constant_values=True)

In [43]:
X_EURUSD = X_EURUSD.loc[X_GBPUSD.index]
X_EURUSD.shape, X_GBPUSD.shape

((160804, 3), (160804, 4))

In [44]:
#para cambiar los valores NaN
X_EURUSD = X_EURUSD.fillna(X_GBPUSD,axis=0)

In [45]:
X_GBPUSD.insert(loc=3,column=3,value=X_EURUSD.iloc[:,0])
X_GBPUSD.insert(loc=4,column=4,value=X_EURUSD.iloc[:,1])
X_GBPUSD.insert(loc=5,column=5,value=X_EURUSD.iloc[:,2])

In [46]:
from sklearn.naive_bayes import GaussianNB
    
y = bt_GBPUSD
n_bdtrain = 4
n_bdtest = 1
mday = 1

est_GNB = PNLEstimatorWrapper(GaussianNB(), PNL_column='PNL')
result_GNB = obj.step_validation(est_GNB, X_GBPUSD, y, obj.v_split(X_GBPUSD,n_bdtrain,n_bdtest,mday))
result_GNB, predict_GNB = obj.redim(result_GNB)
print('--------- GaussianNB ----------------')
print('average PNL : ', np.mean(result_GNB[:,0]))
print('average buy: ', np.mean(result_GNB[:,1]))
print('average sell: ', np.mean(result_GNB[:,2]))
print('-------------------------------------')
print(pd.DataFrame(result_GNB, columns=list(['PNL','buys','sells','longest_buys','longest_sells'])))

--------- GaussianNB ----------------
average PNL :  -0.00199052631579
average buy:  2261.31578947
average sell:  4741.89473684
-------------------------------------
        PNL    buys   sells  longest_buys  longest_sells
0   0.00680  1225.0  5574.0          41.0         2316.0
1   0.00676  3011.0  3822.0        1606.0         2579.0
2   0.00762  3437.0  3611.0         849.0          108.0
3   0.00036  4790.0  2343.0        3361.0         1274.0
4  -0.01519  3032.0  4189.0        2399.0          477.0
5   0.00606  7280.0     0.0        7280.0            0.0
6   0.00038     0.0  7107.0           0.0         7107.0
7  -0.00455     0.0  7515.0           0.0         7515.0
8  -0.00646     0.0  7473.0           0.0         7473.0
9   0.00683     0.0  7361.0           0.0         7361.0
10 -0.01562     0.0  7399.0           0.0         7399.0
11 -0.00114     0.0  7330.0           0.0         7330.0
12 -0.02440     0.0  7579.0           0.0         7579.0
13 -0.00801  6913.0   912.0         